In [1]:
import os, sys
import pandas as pd
import numpy as np
import json

In [2]:
# path 설정
sys.path.append((os.path.abspath("")))
print(sys.path[-1])

d:\base\boostcamp\temp\code


In [3]:
data_path: str = "data/eval/top20"
ease: pd.DataFrame = pd.read_csv(os.path.join(data_path,"recbole_ease_1.csv"))
recvae: pd.DataFrame = pd.read_csv(os.path.join(data_path,"recbole_recvae_2.csv"))
ract: pd.DataFrame = pd.read_csv(os.path.join(data_path,"recbole_ract_subinfo_1.csv"))
bert: pd.DataFrame = pd.read_csv(os.path.join(data_path,"recbole_bert4rec_top20.csv"))
lgcn: pd.DataFrame = pd.read_csv(os.path.join(data_path,"recbole_lightgcn_top20.csv"))


In [ ]:
bert

In [ ]:
lgcn

In [4]:
# 각 데이터프레임에 대해 유저별 순위 매기기
ease['rank'] = ease.groupby('user').cumcount() + 1
recvae['rank'] = recvae.groupby('user').cumcount() + 1  
ract['rank'] = ract.groupby('user').cumcount() + 1
bert['rank'] = bert.groupby('user').cumcount() + 1
lgcn['rank'] = lgcn.groupby('user').cumcount() + 1


In [5]:
import pandas as pd
from collections import defaultdict
from tqdm import tqdm

dataframe_list = [ease, recvae, ract, bert, lgcn]
user_list = dataframe_list[0]['user'].unique()
result = []
tbar = tqdm(user_list, desc='Rank Aggregation with Normalization')
for user in tbar:
    rank_sum = defaultdict(float)
    for df_idx, df in enumerate(dataframe_list):
        items = df[df['user'] == user]['item'].values
        max_rank = min(len(items), 20)
        for rank_idx in range(max_rank):
            # 순위를 정규화하여 합산 (1위는 1, 20위는 0.05)
            normalized_rank = (max_rank - rank_idx) / max_rank
            rank_sum[items[rank_idx]] += normalized_rank
    # 순위의 합이 낮을수록 좋은 아이템 선택
    top_items = sorted(rank_sum.items(), key=lambda x: x[1], reverse=True)[:10]
    for item, _ in top_items:
        result.append((user, item))
submission = pd.DataFrame(result, columns=['user', 'item'])

Rank Aggregation with Normalization: 100%|██████████| 31360/31360 [01:32<00:00, 337.89it/s]


In [10]:
submission.to_csv(os.path.join(data_path,"ensemble_rank_normalization.csv"), index=False)


In [ ]:
# 각 데이터프레임의 인덱스를 점수로 활용하기 위해 순위 컬럼 추가
ease['rank'] = ease.groupby('user').cumcount() 
recvae['rank'] = recvae.groupby('user').cumcount()
ract['rank'] = ract.groupby('user').cumcount()
bert['rank'] = bert.groupby('user').cumcount()
lgcn['rank'] = lgcn.groupby('user').cumcount()

# 각 모델의 추천 결과를 하나로 합치기
merged_df = pd.concat([
    ease.assign(model='ease'),
    recvae.assign(model='recvae'), 
    ract.assign(model='ract'),
    bert.assign(model='bert'),
    lgcn.assign(model='lgcn')
])

# 각 user-item 쌍이 몇 개의 모델에서 추천되었는지 계산
item_counts = merged_df.groupby(['user', 'item']).size().reset_index(name='model_count')

# ease 모델의 순위 정보 가져오기
ease_ranks = ease[['user', 'item', 'rank']].copy()
ease_ranks.columns = ['user', 'item', 'ease_rank']

# item_counts와 ease_ranks 병합
final_scores = item_counts.merge(ease_ranks, on=['user', 'item'], how='left')

# model_count가 높을수록, ease_rank가 낮을수록 좋은 추천으로 정렬
final_recommendations = (final_scores.sort_values(['user', 'model_count', 'ease_rank'], 
                                               ascending=[True, False, True])
                       .groupby('user')
                       .head(10)
                       .sort_values(['user', 'model_count', 'ease_rank'])
                       [['user', 'item', 'model_count']]
                       .reset_index(drop=True))

# 결과 확인
print(f"최종 추천 결과 shape: {final_recommendations.shape}")
final_recommendations


In [ ]:
final_recommendations['model_count'].value_counts()

In [ ]:
final_recommendations[final_recommendations['user']==14]

In [ ]:
final_recommendations = final_recommendations.loc[:,['user','item']]
final_recommendations

In [8]:
rec:pd.DataFrame = pd.read_csv(os.path.join(data_path,"ensemble_ease_recvae_ract_top20.csv"))

In [9]:
# user별로 각 DataFrame의 item이 얼마나 다른지 비교
comparison = pd.merge(rec, submission, on=['user', 'item'], how='outer', indicator=True)

# '_merge' 컬럼의 값에 따라 차이 계산
diff_counts = comparison['_merge'].value_counts()
print("\n데이터프레임 비교 결과:")
print(f"공통 항목: {diff_counts.get('both', 0)}개")
print(f"rec에만 있는 항목: {diff_counts.get('left_only', 0)}개") 
print(f"final_recommendations에만 있는 항목: {diff_counts.get('right_only', 0)}개")

# user별 차이 계산
user_diff = comparison.groupby('user')['_merge'].value_counts().unstack(fill_value=0)
print("\n사용자별 차이:")
print(user_diff)



데이터프레임 비교 결과:
공통 항목: 232399개
rec에만 있는 항목: 81201개
final_recommendations에만 있는 항목: 81201개

사용자별 차이:
_merge  left_only  right_only  both
user                               
11              2           2     8
14              2           2     8
18              2           2     8
25              1           1     9
31              2           2     8
...           ...         ...   ...
138473          1           1     9
138475          2           2     8
138486          3           3     7
138492          3           3     7
138493          3           3     7

[31360 rows x 3 columns]
